In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [4]:
X = train[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y = train[['Survived']]

In [5]:
 X = pd.concat([X, pd.get_dummies(X['Sex'])], axis=1)

In [6]:
del X['Sex']

In [7]:
X = pd.concat([X,pd.get_dummies(X['Embarked'])], axis=1)

In [8]:
del X['Embarked']

In [9]:
X['Age'].fillna(X['Age'].mean(), inplace=True)

In [10]:
X = pd.concat([X,pd.get_dummies(X['Pclass'])], axis=1)

In [11]:
del X['Pclass']

In [12]:
#X.rename(columns={'1':'Pclass1','2':'Pclass2','3':'Pclass3'}, inplace=True)
X = X.rename(columns={1:'Pclass1',2:'Pclass2',3:'Pclass3'})

In [13]:
#from sklearn.cross_validation import train_test_split 

In [14]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(class_weight='balanced')

In [16]:
from scipy.stats import randint as sp_randint
from sklearn.grid_search import RandomizedSearchCV
param_dist = {"max_depth": [3,4,5,6,7, None],
              "max_features": sp_randint(2, 12),
              "min_samples_split": sp_randint(2, 12),
              "min_samples_leaf": sp_randint(2, 12),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
optimised_model = RandomizedSearchCV(model,  param_distributions=param_dist)

In [17]:
optimised_model.fit(np.array(X),y['Survived'])

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10768b410>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10768b390>, 'criterion': ['gini', 'entropy'], 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10768b210>, 'max_depth': [3, 4, 5, 6, 7, None]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=None, verbose=0)

In [18]:
best_parameters = optimised_model.best_estimator_.get_params()
for param_name in sorted(param_dist.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

	bootstrap: False
	criterion: 'gini'
	max_depth: None
	max_features: 8
	min_samples_leaf: 8
	min_samples_split: 8


In [19]:
optimised_model = optimised_model.best_estimator_

In [20]:
#y_predict = optimised_model.predict(np.array(X_test))

In [21]:
#from sklearn.metrics import confusion_matrix
#confusion_matrix(np.array(y_test),y_predict)

In [22]:
optimised_model.score(X,y)

0.89113355780022452

In [23]:
#optimised_model.score(X_train,y_train)

In [24]:
#optimised_model.score(X_test,y_test)

In [25]:
test = pd.read_csv('test.csv')

In [26]:
X = test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]

In [27]:
X = pd.concat([X, pd.get_dummies(X['Sex'])], axis=1)

In [28]:
del X['Sex']

In [29]:
X = pd.concat([X, pd.get_dummies(X['Embarked'])], axis=1)

In [30]:
del X['Embarked']

In [31]:
X = pd.concat([X, pd.get_dummies(X['Pclass'])], axis=1)

In [32]:
del X['Pclass']

In [33]:
X = X.rename(columns={1:'Pclass1',2:'Pclass2',3:'Pclass3'})

In [34]:
X['Age'].fillna(X['Age'].mean(), inplace=True)

In [35]:
X['Fare'].fillna(0, inplace=True)

In [36]:
y_predict = optimised_model.predict(X)

In [37]:
d = {'PassengerId' : test['PassengerId'], 'Survived':y_predict}
submission = pd.DataFrame(d)

In [38]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [40]:
submission.to_csv('randomforestsubmission2.csv',index=False)